<div style="
    background: linear-gradient(135deg, #6e8efb, #a777e3);
    padding: 30px;
    border-radius: 15px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.2);
    text-align: center;
    color: white;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
    <h1 style="margin: 0; font-size: 40px; border: none; color: white;">Анализ личности клиента</h1>
    <p style="margin: 10px 0 0; font-size: 18px; opacity: 0.9;">Анализ данных, Кластеризация и создание моделей (+ Интерактивный Дашбоард)</p>
</div>